### ВЫПОЛНИМ РАСПРЕДЕЛЕННОЕ ОБУЧЕНИЕ НЕЙРОСЕТИ С PYSPARK И KERAS

Данильченко Вадим

In [1]:
# Устанавливаем необходимые компоненты: Java, spark и findspark
!apt-get install openjdk-8-jdk-headless -qq #> /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.7.tgz
!tar -xf spark-2.4.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.3-bin-hadoop2.7"

In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [3]:
spark.version

'2.4.3'

In [27]:
# !pip install elephas

# внесем исправления в библиотеку elephas, чтобы брались корректные параметры модели
# model.compiled_metrics._metrics меняем на model.metrics_names

!pip uninstall elephas
# !git clone https://github.com/maxpumperla/elephas.git
!pip3 install -U /content/elephas/

Found existing installation: elephas 3.0.0
Uninstalling elephas-3.0.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/docs/*
    /usr/local/lib/python3.7/dist-packages/elephas-3.0.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/elephas/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/docs/md_autogen.py
    /usr/local/lib/python3.7/dist-packages/docs/update_docs.py
Proceed (y/n)? y
  Successfully uninstalled elephas-3.0.0
Processing ./elephas
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for elephas: filename=elephas-3.0.0-py3-none-any.whl size=27084 sha256=81e22dd1

In [6]:
# установим tensorflow==2.0.0, без него ошибка "AttributeError: 'Sequential' object has no attribute 'compiled_metrics'""
# !pip install tensorflow==2.0.0

In [4]:
# импортируем библиотеки

from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.ml.feature import OneHotEncoder, StandardScaler, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.optimizers import serialize as serialize_optimizer
from tensorflow.keras import optimizers, regularizers

from elephas.spark_model import SparkModel
from elephas.ml_model import ElephasEstimator

from tqdm import tqdm

In [6]:
# скачаем датасет Kaggle-Ames-Housing-Prices
!wget https://raw.githubusercontent.com/hjhuney/Kaggle-Ames-Housing-Prices/master/data/train.csv

--2021-10-13 14:14:31--  https://raw.githubusercontent.com/hjhuney/Kaggle-Ames-Housing-Prices/master/data/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460676 (450K) [text/plain]
Saving to: ‘train.csv.4’

train.csv.4         100%[===================>] 449.88K  --.-KB/s    in 0.03s   

2021-10-13 14:14:31 (15.1 MB/s) - ‘train.csv.4’ saved [460676/460676]



In [7]:
!ls

elephas			       spark-2.4.3-bin-hadoop2.7.tgz.1	train.csv.2
sample_data		       spark-warehouse			train.csv.3
spark-2.4.3-bin-hadoop2.7      train.csv			train.csv.4
spark-2.4.3-bin-hadoop2.7.tgz  train.csv.1


In [5]:
# загрузим датасет
df = spark.read.csv('train.csv', header=True, inferSchema=True)
df.show()

+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+---------+
| Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition

In [6]:
# заполним все na значениями

# None
cols_na2none = ['Alley', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'BsmtQual', 'Exterior2nd', 'Fence', 'FireplaceQu', 'GarageCond', 'GarageFinish', 
                'GarageQual', 'GarageType', 'MasVnrType', 'MiscFeature', 'PoolQC', ]

for col_ in cols_na2none:
    df = df.na.fill({col_:"None"})

# 0
cols_na2zero = ['BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', 'GarageArea', 'GarageCars', 'GarageYrBlt', 'LotFrontage', 'MasVnrArea']    

for col_ in cols_na2zero:
    df = df.na.fill({col_:0})
    
# SBrkr
df = df.na.fill({"Electrical": "SBrkr"})

# VinylSd
df = df.na.fill({"Exterior1st": "VinylSd"})

# Typ
df = df.na.fill({"Functional": "Typ"})

# TA
df = df.na.fill({"KitchenQual": "TA"})

# RL
df = df.na.fill({"MSZoning": "RL"})

# WD
df = df.na.fill({"SaleType": "WD"})

# AllPub
df = df.na.fill({"Utilities": "AllPub"})

In [7]:
# проверим какое-нибудь поле
df.select("Alley").describe().show()

+-------+-----+
|summary|Alley|
+-------+-----+
|  count| 1460|
|   mean| null|
| stddev| null|
|    min| Grvl|
|    max| Pave|
+-------+-----+



In [8]:
# сделаем необходимые замены значений
df = df.replace("Grvl", '1', subset=['Alley']).replace("Pave", '2', subset=['Alley']).replace("NA", '0', subset=['Alley'])
df = df.replace("Po", '1', subset=['BsmtCond']).replace("Fa", '2', subset=['BsmtCond']).replace("TA", '3', subset=['BsmtCond']).replace("Gd", '4', subset=['BsmtCond']) \
    .replace("Ex", '5', subset=['BsmtCond']).replace("NA", '0', subset=['BsmtCond'])
df = df.replace("No", '2', subset=['BsmtExposure']).replace("Mn", '2', subset=['BsmtExposure']).replace("Av", '3', subset=['BsmtExposure']).replace("Gd", '4', subset=['BsmtExposure']).replace("NA", '0', subset=['BsmtExposure'])
df = df.replace("Unf", '1', subset=['BsmtFinType1']).replace("LwQ", '2', subset=['BsmtFinType1']).replace("Rec", '3', subset=['BsmtFinType1']).replace("BLQ", '4', subset=['BsmtFinType1']) \
    .replace("ALQ", '5', subset=['BsmtFinType1']).replace("GLQ", '6', subset=['BsmtFinType1']).replace("NA", '0', subset=['BsmtFinType1'])
df = df.replace("Unf", '1', subset=['BsmtFinType2']).replace("LwQ", '2', subset=['BsmtFinType2']).replace("Rec", '3', subset=['BsmtFinType2']).replace("BLQ", '4', subset=['BsmtFinType2']) \
    .replace("ALQ", '5', subset=['BsmtFinType2']).replace("GLQ", '6', subset=['BsmtFinType2']).replace("NA", '0', subset=['BsmtFinType2'])
df = df.replace("Po", '1', subset=['BsmtQual']).replace("Fa", '2', subset=['BsmtQual']).replace("TA", '3', subset=['BsmtQual']).replace("Gd", '4', subset=['BsmtQual']) \
    .replace("Ex", '5', subset=['BsmtQual']).replace("NA", '0', subset=['BsmtQual'])
df = df.replace("N", '1', subset=['CentralAir']).replace("Y", '2', subset=['CentralAir']).replace("NA", '0', subset=['CentralAir'])
df = df.replace("Po", '1', subset=['ExterCond']).replace("Fa", '2', subset=['ExterCond']).replace("TA", '3', subset=['ExterCond']).replace("Gd", '4', subset=['ExterCond']) \
    .replace("Ex", '5', subset=['ExterCond']).replace("NA", '0', subset=['ExterCond'])
df = df.replace("Po", '1', subset=['ExterQual']).replace("Fa", '2', subset=['ExterQual']).replace("TA", '3', subset=['ExterQual']).replace("Gd", '4', subset=['ExterQual']) \
    .replace("Ex", '5', subset=['ExterQual']).replace("NA", '0', subset=['ExterQual'])
df = df.replace("MnWw", '1', subset=['Fence']).replace("GdWo", '2', subset=['Fence']).replace("MnPrv", '3', subset=['Fence']).replace("GdPrv", '4', subset=['Fence']).replace("NA", '0', subset=['Fence'])
df = df.replace("Po", '1', subset=['FireplaceQu']).replace("Fa", '2', subset=['FireplaceQu']).replace("TA", '3', subset=['FireplaceQu']).replace("Gd", '4', subset=['FireplaceQu']) \
    .replace("Ex", '5', subset=['FireplaceQu']).replace("NA", '0', subset=['FireplaceQu'])
df = df.replace("Sal", '1', subset=['Functional']).replace("Sev", '2', subset=['Functional']).replace("Maj2", '3', subset=['Functional']).replace("Maj1", '4', subset=['Functional']) \
    .replace("Mod", '5', subset=['Functional']).replace("Min2", '6', subset=['Functional']).replace("Min1", '7', subset=['Functional']).replace("Typ", '8', subset=['Functional']).replace("NA", '0', subset=['Functional'])
df = df.replace("Po", '1', subset=['GarageCond']).replace("Fa", '2', subset=['GarageCond']).replace("TA", '3', subset=['GarageCond']).replace("Gd", '4', subset=['GarageCond']) \
    .replace("Ex", '5', subset=['GarageCond']).replace("NA", '0', subset=['GarageCond'])
df = df.replace("Po", '1', subset=['GarageQual']).replace("Fa", '2', subset=['GarageQual']).replace("TA", '3', subset=['GarageQual']).replace("Gd", '4', subset=['GarageQual']) \
    .replace("Ex", '5', subset=['GarageQual']).replace("NA", '0', subset=['GarageQual'])
df = df.replace("Unf", '1', subset=['GarageFinish']).replace("RFn", '2', subset=['GarageFinish']).replace("Fin", '3', subset=['GarageFinish']).replace("NA", '0', subset=['GarageFinish'])
df = df.replace("Po", '1', subset=['HeatingQC']).replace("Fa", '2', subset=['HeatingQC']).replace("TA", '3', subset=['HeatingQC']).replace("Gd", '4', subset=['HeatingQC']) \
    .replace("Ex", '5', subset=['HeatingQC']).replace("NA", '0', subset=['HeatingQC'])
df = df.replace("Po", '1', subset=['KitchenQual']).replace("Fa", '2', subset=['KitchenQual']).replace("TA", '3', subset=['KitchenQual']).replace("Gd", '4', subset=['KitchenQual']) \
    .replace("Ex", '5', subset=['KitchenQual']).replace("NA", '0', subset=['KitchenQual'])
df = df.replace("Low", '1', subset=['LandContour']).replace("HLS", '2', subset=['LandContour']).replace("Bnk", '3', subset=['LandContour']).replace("Lvl", '4', subset=['LandContour']).replace("NA", '0', subset=['LandContour'])
df = df.replace("Sev", '1', subset=['LandSlope']).replace("Mod", '2', subset=['LandSlope']).replace("Gtl", '3', subset=['LandSlope']).replace("NA", '0', subset=['LandSlope'])
df = df.replace("IR3", '1', subset=['LotShape']).replace("IR2", '2', subset=['LotShape']).replace("IR1", '3', subset=['LotShape']).replace("Reg", '4', subset=['LotShape']).replace("NA", '0', subset=['LotShape'])
df = df.replace("N", '0', subset=['PavedDrive']).replace("P", '1', subset=['PavedDrive']).replace("Y", '2', subset=['PavedDrive']).replace("NA", '0', subset=['PavedDrive'])
df = df.replace("Fa", '1', subset=['PoolQC']).replace("TA", '2', subset=['PoolQC']).replace("Gd", '3', subset=['PoolQC']).replace("Ex", '4', subset=['PoolQC']).replace("NA", '0', subset=['PoolQC'])
df = df.replace("Grvl", '1', subset=['Street']).replace("Pave", '2', subset=['Street']).replace("NA", '0', subset=['Street'])
df = df.replace("ELO", '1', subset=['Utilities']).replace("NoSeWa", '2', subset=['Utilities']).replace("NoSewr", '3', subset=['Utilities']).replace("AllPub", '4', subset=['Utilities']).replace("NA", '0', subset=['Utilities'])

# проверим какое-нибудь поле
df.select("Alley").describe().show()

+-------+-------------------+
|summary|              Alley|
+-------+-------------------+
|  count|               1460|
|   mean|0.09041095890410959|
| stddev| 0.3721505895356691|
|    min|                  0|
|    max|                  2|
+-------+-------------------+



In [9]:
# присвоим номер сезона новому полю в зависимости от месяца
seasons = {12 : 0, 1 : 0, 2 : 0, 
           3 : 1, 4 : 1, 5 : 1,
           6 : 2, 7 : 2, 8 : 2, 
           9 : 3, 10 : 3, 11 : 3}

def check_seasons(month_):
    return seasons[month_]


udf_seasons = udf(lambda x: check_seasons(x), returnType=StringType())

df = df.withColumn("SeasonSold", udf_seasons(col("MoSold")))

# проверим результат
df.select(['MoSold', 'SeasonSold']).show(5)

+------+----------+
|MoSold|SeasonSold|
+------+----------+
|     2|         0|
|     5|         1|
|     9|         3|
|     2|         0|
|    12|         0|
+------+----------+
only showing top 5 rows



In [10]:
# посчитаем количество лет
df = df.withColumn("YrActualAge", col("YrSold")-col("YearBuilt"))

# проверим результат
df.select(['YrSold', 'YearBuilt', 'YrActualAge']).show(5)

+------+---------+-----------+
|YrSold|YearBuilt|YrActualAge|
+------+---------+-----------+
|  2008|     2003|          5|
|  2007|     1976|         31|
|  2008|     2001|          7|
|  2006|     1915|         91|
|  2008|     2000|          8|
+------+---------+-----------+
only showing top 5 rows



In [11]:
# проведем дополнительные вычисления
df = df.withColumn("TotalSF1", col("TotalBsmtSF")+col("1stFlrSF")+col("2ndFlrSF"))
df = df.withColumn("TotalSF2", col("BsmtFinSF1")+col("BsmtFinSF2")+col("1stFlrSF")+col("2ndFlrSF"))
df = df.withColumn("AllSF", col("GrLivArea")+col("TotalBsmtSF"))
df = df.withColumn("AllFlrsSF", col("1stFlrSF")+col("2ndFlrSF"))
df = df.withColumn("AllPorchSF", col("OpenPorchSF")+col("EnclosedPorch")+col("3SsnPorch")+col("ScreenPorch"))
df = df.withColumn("TotalBath", 2*(col("FullBath")+0.5*col("HalfBath")+col("BsmtFullBath")+0.5*col("BsmtHalfBath")))

# конвертируем в integer
df = df.withColumn("TotalBath", col("TotalBath").cast(IntegerType()))

df = df.withColumn("TotalPorch", col("OpenPorchSF")+col("3SsnPorch")+col("EnclosedPorch")+col("ScreenPorch")+col("WoodDeckSF"))
df = df.withColumn("OverallScore", col("OverallQual")*col("OverallCond"))
df = df.withColumn("GarageScore", col("GarageQual")*col("GarageCond"))
df = df.withColumn("ExterScore", col("ExterQual")*col("ExterCond"))
df = df.withColumn("KitchenScore", col("KitchenAbvGr")*col("KitchenQual"))
df = df.withColumn("FireplaceScore", col("Fireplaces")*col("FireplaceQu"))
df = df.withColumn("GarageScore", col("GarageArea")*col("GarageQual"))
df = df.withColumn("PoolScore", col("PoolArea")*col("PoolQC"))

# применим конструкцию условного оператора
df = df.withColumn("hasPool", when(col("PoolArea")>0,1).otherwise(0))
df = df.withColumn("has2ndFloor", when(col("2ndFlrSF")>0,1).otherwise(0))
df = df.withColumn("hasGarage", when(col("GarageArea")>0,1).otherwise(0))
df = df.withColumn("hasBsmt", when(col("TotalBsmtSF")>0,1).otherwise(0))
df = df.withColumn("hasFireplace", when(col("Fireplaces")>0,1).otherwise(0))

In [12]:
# создадим пайплайн обработки фичей

# функция возвращаеттип поля
def get_dtype(df,colname):
    return [dtype for name, dtype in df.dtypes if name == colname][0]

# проверим какой тип у поля и запишем в нужный список
label = 'SalePrice'
num_features = [] # числовые
cat_features = [] # категориальные
for colname in df.columns:
    if colname!=label:
      if get_dtype(df, colname) != 'string':
          num_features.append(str(colname))
      else:
          cat_features.append(str(colname))

pipeline_stages = []

# обработаем категориальные фичи
for f in cat_features:
  str_idx = StringIndexer(inputCol=f,outputCol=f+'_idx')
  encoder = OneHotEncoder(inputCol=str_idx.getOutputCol(), outputCol=f+'_ohe')
  pipeline_stages.extend([str_idx,encoder])

# обработаем числовые фичи
features = VectorAssembler(inputCols=num_features, outputCol='features')
scaler = StandardScaler(inputCol='features',outputCol='features_scaled')
pipeline_stages.extend([features,scaler])

joined_features = [feature+'_ohe' for feature in cat_features]
joined_features.append('features_scaled')
joined_stage = VectorAssembler(inputCols=joined_features, outputCol='features_all')
pipeline_stages.extend([joined_stage])

In [13]:
# выведем этапы пайплайна
pipeline_stages

[StringIndexer_86da3f188815,
 OneHotEncoder_50d6b467cc86,
 StringIndexer_8e2af566460c,
 OneHotEncoder_c892f3b9d9d2,
 StringIndexer_3931d4f5042c,
 OneHotEncoder_191549b9c289,
 StringIndexer_1e705eb2e43b,
 OneHotEncoder_ed59601db336,
 StringIndexer_73ff4f149770,
 OneHotEncoder_e87f855c661b,
 StringIndexer_ac51afdb7456,
 OneHotEncoder_62c86cf1764c,
 StringIndexer_7ed6d903c578,
 OneHotEncoder_a5641e2bbfff,
 StringIndexer_ee3286679782,
 OneHotEncoder_8d6e48503db7,
 StringIndexer_c7a3cf2711b9,
 OneHotEncoder_338db5ab8da8,
 StringIndexer_d8ea1bf39301,
 OneHotEncoder_efc1fc5f7d16,
 StringIndexer_6756ae39b4c3,
 OneHotEncoder_ad0affb15149,
 StringIndexer_a7db68f50125,
 OneHotEncoder_848787bd9726,
 StringIndexer_77be949fe105,
 OneHotEncoder_60d5c7ff8e1a,
 StringIndexer_4eceb3784eb3,
 OneHotEncoder_5f93994526a1,
 StringIndexer_89961542614d,
 OneHotEncoder_e8753953fc06,
 StringIndexer_a1745bae83be,
 OneHotEncoder_61b09e6d40f8,
 StringIndexer_b12b7375b655,
 OneHotEncoder_55aa293ecb04,
 StringIndexer

In [14]:
# выполним преобразования согласно заданному пайплайну
pipeline = Pipeline(stages = pipeline_stages)
pipeline_fitted = pipeline.fit(df)
df_transformed = pipeline_fitted.transform(df)

In [15]:
# посмотрим на получившийся датафрейм
df_transformed.show()

+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+---------+----------+-----------+--------+--------+-----+---------+----------+---------+----------+------------+-----------+----------+------------+-----

In [16]:
# разобьем на обучающую и тестовую выборки
train,test = df_transformed.selectExpr('features_all as features', label).randomSplit([0.8,0.2])

print("train:", train.count())
print("test:", test.count())

train.show(3)
test.show(3)

train: 1161
test: 299
+--------------------+---------+
|            features|SalePrice|
+--------------------+---------+
|(814,[0,4,114,115...|   144000|
|(814,[0,4,114,115...|    87500|
|(814,[0,4,114,115...|   180000|
+--------------------+---------+
only showing top 3 rows

+--------------------+---------+
|            features|SalePrice|
+--------------------+---------+
|(814,[0,4,114,115...|   228500|
|(814,[0,4,114,115...|   135000|
|(814,[0,4,114,115...|   139000|
+--------------------+---------+
only showing top 3 rows



In [17]:
train.limit(5).toPandas()

,features,SalePrice
0,"(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",144000
1,"(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",87500
2,"(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",180000
3,"(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",272000
4,"(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",230000


In [25]:
# опишем архитектуру нейросетки

n_features = len(train.select('features').first()[0])
print("количество фичей:", n_features)

model = Sequential()
model.add(Dense(128, input_shape = (n_features,)))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(1))
model.add(Activation('linear'))
model.compile(loss='mse', optimizer=Adam(lr=0.01))

количество фичей: 814


In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               104320    
_________________________________________________________________
activation_3 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
activation_4 (Activation)    (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
_________________________________________________________________
activation_5 (Activation)    (None, 1)                

In [23]:
num_workers = 2
optimizer_conf = Adam(lr=0.01)
opt_conf = optimizers.serialize(optimizer_conf)

estimator = ElephasEstimator()
estimator.setFeaturesCol('features')
estimator.setLabelCol('prediction')
estimator.set_keras_model_config(model.to_yaml())
estimator.set_categorical_labels(True)
estimator.set_nb_classes(1)
estimator.set_num_workers(num_workers)
estimator.set_epochs(15)
estimator.set_batch_size(64)
estimator.set_verbosity(1)
estimator.set_validation_split(0.1)
estimator.set_optimizer_config(opt_conf)
estimator.set_mode('synchronous')
estimator.set_loss('mse')

ElephasEstimator_87a7ecf3d8bb

In [30]:
dl_pipeline = Pipeline(stages=[estimator])
dl_train = dl_pipeline.fit(train.selectExpr("features", "SalePrice as prediction"))
pred = dl_train.transform(test.selectExpr("features"))

>>> Fit model
>>> Synchronous training complete.
